In [13]:
#-*-coding:utf-8-*-
import itertools as iters
from time import time
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.cross_validation import KFold
from sklearn.feature_selection import f_classif
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
import matplotlib.pylab as plt
from math import log2 
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

def entropy(data):
    length,dataDict=len(data),{} 
    for b in data:  
        try:dataDict[b]+=1  
        except:dataDict[b]=1  
    entropy=sum([-d/length*log2(d/length) for d in list(dataDict.values())])
    return entropy

def informationgain(data,label):
    informationgain = []
    la = entropy(label)
    print(la)
    for j in range(data.shape[1] ):
        feature = data[:,j]
        for a in set(feature):
            ent = []
            op = []
            ne = []
            for k in range(len(feature)):
                if feature[k] >= a:
                    op.append(label[k])
                else:
                    ne.append(label[k])
            if len(op) == 0 or len(ne) == 0:
                ent.append(la)
            else:
                ent.append(len(op)*entropy(op)/len(label) + len(ne)*entropy(ne)/len(label))
        informationgain.append(la-min(ent))
    return informationgain

##计算三肽特征
def statisPsi_3(seqs,protein,gap1,gap2):
    psi = np.zeros(len(seqs))
    loops = len(protein) - gap1 - gap2 - 2
    for start in range(loops):
        dipeptide = protein[start] + protein[start + gap1 + 1] + protein[start + 2 + gap1 + gap2]
        index = seqs.index(dipeptide)
        psi[index] += 1
    psi = np.array(psi)
    psi = psi / sum(psi)
    return psi

# get gap dipeptide features psi matrix",
def all_psi(dataset,gap1,gap2):
    gap_psi = np.zeros((len(dataset), len(DIPEPTIDE)))
    for idx in range(len(dataset)):
        gap_psi[idx] = statisPsi_3(DIPEPTIDE, dataset[idx], gap1,gap2)
    return gap_psi

#输入核函数名称和参数gamma值，返回SVM训练十折交叉验证的准确率
def SVM_10fold(data,label):
    ##数据归一化（按列处理，必须做）
    scaler = preprocessing.StandardScaler().fit(data)
    data = scaler.transform(data)
    kf = KFold(data.shape[0],n_folds=10,shuffle = True)  #固定随机数可固定分组
    precision_average = 0.0
    TPCount = 0
    TNCount = 0
    train = []
    test = []
    test_true = []

    C_range = np.logspace(15, 5, 11, base=2)
    gamma_range = np.logspace(-15, -25, 11, base=2)
    param_grid = dict(gamma=gamma_range, C=C_range)
    grid = GridSearchCV(SVC(kernel='rbf'), param_grid=param_grid, cv=5)  # 基于交叉验证的网格搜索

    for train_index, test_index in kf:
        grid = grid.fit(data[train_index],label[train_index])
        clf = SVC(C = grid.best_params_['C'],gamma=grid.best_params_['gamma'])
        clf.fit(data[train_index],label[train_index])
        pred_train = clf.predict(data[train_index]).tolist()
        pred_test = clf.predict(data[test_index]).tolist()

        train.append(pred_train)
        test = test + pred_test
        test_true = test_true + label[test_index].tolist()

        testLabel = label[test_index]

        # print(metrics.confusion_matrix(pred_train,trainLabel))
        TP = 0
        TN = 0
        #同时输出svm的结果
        #查看测试集的结果
        for i in range(len(pred_test)):
            if pred_test[i] == 1 and (testLabel[i] == 1):
                TP = TP + 1
            elif pred_test[i] == 0 and (testLabel[i] == 0):
                TN = TN + 1
        TPCount = TPCount + TP
        TNCount = TNCount + TN

        precision = (TP + TN) * 1.0 / len(testLabel)
        precision_average = precision_average + precision


    precision_average = precision_average / 10
    positiveReca1 = TPCount * 1.0 / 99
    negtiveReca1 = TNCount * 1.0 / 208
    MCC = matthews_corrcoef(test_true,test)


    print (u'SVM的测试集结果：')
    print (TPCount,TNCount)
    print (u'准确率为' + str(precision_average))
    print (u'正类召回率为' + str(positiveReca1))
    print (u'负类召回率为' + str(negtiveReca1))
    print(u'MCC为' + str(matthews_corrcoef(test_true,test)))
    print(accuracy_score(test_true,test))


    # ##保存最优模型的训练、测试集结果
    # with open('C:\\Users\Administrator\Desktop\data\\result\\result_4gapTC_svm_train.txt','w') as file:
    #     for i in train:
    #         file.write(str(i) + '\n')
    # with open('C:\\Users\Administrator\Desktop\data\\result\\result_4gapTC_svm_test.txt','w') as file:
    #     for i in test:
    #         file.write(str(i) + '\n')


    return precision_average,positiveReca1,negtiveReca1,MCC

if(__name__=='__main__'):
    # t0 = time()
    path = r''
    with open('virion.txt', "r") as file:
        train_tdata = [line.strip() for line in file if '>' != line[0]]
    with open( 'non-virion.txt', "r") as file:
        train_fdata = [line.strip() for line in file if '>' != line[0]]

    SAA = ('ACDEFGHIKLMNPQRSTVWY')
    DIPEPTIDE = []

    for dipeptide in iters.product(SAA, repeat=3):
        DIPEPTIDE.append(''.join(dipeptide))

    label = pd.Series([1 for i in range(len(train_tdata))]+ [0 for i in range(len(train_fdata))])
    label = label.as_matrix()

    ##将序列处理为特征向量
    ##gap1 和 gap2 的数值是任意选取的，一般是从0到10遍历寻优的，但是那样太浪费时间了，你可以先直接指定一个值先看效果
    gap1 = 3
    gap2 = 5
    gap_T = all_psi(train_tdata,gap1,gap2)
    gap_F = all_psi(train_fdata,gap1,gap2)

    dataAll = np.row_stack((gap_T, gap_F))  # 矩阵按行合并
    # print(data.shape)
    ##将特征结果保存，方便下次直接读取
    # np.savetxt('E:\Python Program\Protein\data\gapAll\\virion_gap0_dipe2.csv', data, delimiter=',')
    ##直接读入特征数据
    # dataAll = pd.read_csv(path = '\\virion_gap0_dipe2.csv',header=None).as_matrix()


    print(dataAll.shape)
    ##计算F值
    f = f_classif(dataAll,label)[0]

    ##因为三肽特征较为稀疏，会有一些全为nan的列，要对这些列进行处理
    a = np.array(f)
    nan_count = np.sum(a != a)
    print(nan_count)
    ##将nan替换为0
    f1 = np.nan_to_num(a)
    ##将F值从大到小排序，获得相应的位置序号
    f_order = np.argsort(f1).tolist()[::-1]
    data = dataAll[:,f_order[0:721]]
    

    for cishu in range(130):
        presion,positiveRecall,negtiveRecall,matthews = SVM_10fold(data,label)
        acc.append(presion)
        pr.append(positiveRecall)
        nr.append(negtiveRecall)
        MCC.append(matthews)
        print(cishu)


    #选取所需的特征维度，构造模型，如这里是前100维特征，这个数值按照需要也可以更改
    



    ###为了选取最优特征子集，一般需要遍历，结果运行、保存、展示如下
#     p = []
#     for i in range(1,800,10):
#         data = dataAll[:, f_order[0:i]]
#         print(data.shape)
#         presion, positiveRecall, negtiveRecall = SVM_10fold(data, label)
#         p.append(presion)
#     # print(SVM_10fold(data,label))
#     np.savetxt(str(gap1) + 'and' + str(gap2) + 'informationgainstatisPsi_3.csv',p,delimiter=',')
#     plt.plot(range(len(p)),p)
#     plt.savefig(str(gap1) +'and' + str(gap2) + 'informationgainstatisPsi_3.pdf')
#     plt.close()


(307, 8000)
455


D:\softwares\Anaconda\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [  26   90  138  201  218  221  321  361  366  371  406  410  418  421  423
  425  426  429  430  431  432  433  434  435  438  439  441  450  459  461
  470  477  481  483  484  490  498  518  522  526  531  532  533  534  535
  536  537  538  539  541  552  555  557  558  560  561  570  573  575  593
  598  601  602  604  605  606  607  608  609  610  613  614  616  618  619
  622  635  636  638  639  646  648  659  661  665  666  670  673  676  681
  690  693  697  699  701  702  706  727  731  746  754  761  763  767  769
  770  771  772  773  775  776  779  781  785  786  790  793  798  799  826
  838  930 1001 1010 1018 1221 1236 1238 1258 1321 1326 1330 1339 1561 1621
 1626 1630 1631 1633 1638 1690 1738 1801 1806 1821 1841 1846 1861 1881 1966
 1970 1972 1978 1981 2021 2030 2036 2039 2138 2181 2201 2401 2406 2420 2424
 2426 2427 2429 2430 2432 2435 2438 2441 2446 2

SVM的测试集结果：
90 206
准确率为0.9644086021505377
正类召回率为0.9090909090909091
负类召回率为0.9903846153846154
MCC为0.917752278619
0.964169381107
0
SVM的测试集结果：
91 207
准确率为0.970752688172043
正类召回率为0.9191919191919192
负类召回率为0.9951923076923077
MCC为0.932963916106
0.970684039088
1
SVM的测试集结果：
90 206
准确率为0.9641935483870968
正类召回率为0.9090909090909091
负类召回率为0.9903846153846154
MCC为0.917752278619
0.964169381107
2
SVM的测试集结果：
90 207
准确率为0.9675268817204301
正类召回率为0.9090909090909091
负类召回率为0.9951923076923077
MCC为0.92556338044
0.967426710098
3
SVM的测试集结果：
90 206
准确率为0.9643010752688171
正类召回率为0.9090909090909091
负类召回率为0.9903846153846154
MCC为0.917752278619
0.964169381107
4
SVM的测试集结果：
92 206
准确率为0.9706451612903226
正类召回率为0.9292929292929293
负类召回率为0.9903846153846154
MCC为0.932676681752
0.970684039088
5
SVM的测试集结果：
87 206
准确率为0.9540860215053764
正类召回率为0.8787878787878788
负类召回率为0.9903846153846154
MCC为0.895430717886
0.954397394137
6
SVM的测试集结果：
88 206
准确率为0.957741935483871
正类召回率为0.8888888888888888
负类召回率为0.9903846153846154
MCC为0.902863569488
0.95

SVM的测试集结果：
92 205
准确率为0.9672043010752688
正类召回率为0.9292929292929293
负类召回率为0.9855769230769231
MCC为0.92507905534
0.967426710098
65
SVM的测试集结果：
89 207
准确率为0.9641935483870968
正类召回率为0.898989898989899
负类召回率为0.9951923076923077
MCC为0.91817225809
0.964169381107
66
SVM的测试集结果：
87 205
准确率为0.9508602150537635
正类召回率为0.8787878787878788
负类召回率为0.9855769230769231
MCC为0.887554880886
0.951140065147
67
SVM的测试集结果：
91 206
准确率为0.9670967741935484
正类召回率为0.9191919191919192
负类召回率为0.9903846153846154
MCC为0.925209645313
0.967426710098
68
SVM的测试集结果：
90 205
准确率为0.9608602150537635
正类召回率为0.9090909090909091
负类召回率为0.9855769230769231
MCC为0.91004470345
0.960912052117
69
SVM的测试集结果：
87 206
准确率为0.9541935483870969
正类召回率为0.8787878787878788
负类召回率为0.9903846153846154
MCC为0.895430717886
0.954397394137
70
SVM的测试集结果：
87 206
准确率为0.954516129032258
正类召回率为0.8787878787878788
负类召回率为0.9903846153846154
MCC为0.895430717886
0.954397394137
71
SVM的测试集结果：
92 206
准确率为0.970752688172043
正类召回率为0.9292929292929293
负类召回率为0.9903846153846154
MCC为0.932676681752


In [17]:
print(len(MCC))
print(np.mean(MCC))
a = np.array(MCC)

np.savetxt('./3anova/MCC.csv',a, delimiter = ',')

200
0.913547046482
